In [2]:
import boto3

reko = boto3.client('rekognition')

In [3]:
bucket = 'datasets-dggallit'
video = 'rekognition-video-demo/video-test.mp4'

response = reko.start_face_detection(
    Video={'S3Object': {'Bucket': bucket, 'Name': video}}
)
response

{'JobId': 'e03901696a57596cec2840e3358a3d3246a97c0fc8c6b1c608942e798b8c9d52',
 'ResponseMetadata': {'RequestId': '4d27cf4f-db15-422a-9d3c-864046b84393',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Thu, 03 Jun 2021 18:15:59 GMT',
   'x-amzn-requestid': '4d27cf4f-db15-422a-9d3c-864046b84393',
   'content-length': '76',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

In [4]:
import time

job_id = response['JobId']
response = reko.get_face_detection(JobId=job_id)

while (response['JobStatus'] == 'IN_PROGRESS'):
    time.sleep(30)
    response = reko.get_face_detection(JobId=job_id)

In [10]:
response['Faces']

 'Y': 0.5602668523788452},
     {'Type': 'eyeRight', 'X': 0.698707640171051, 'Y': 0.5605025887489319},
     {'Type': 'mouthLeft', 'X': 0.6750101447105408, 'Y': 0.6109495162963867},
     {'Type': 'mouthRight', 'X': 0.696586012840271, 'Y': 0.6111142039299011},
     {'Type': 'nose', 'X': 0.6861732006072998, 'Y': 0.5868691802024841}],
    'Pose': {'Roll': -1.0739846229553223,
     'Yaw': 0.727107584476471,
     'Pitch': 2.4473981857299805},
    'Quality': {'Brightness': 93.07357788085938,
     'Sharpness': 26.1773681640625},
    'Confidence': 99.99927520751953}},
  {'Timestamp': 72540,
   'Face': {'BoundingBox': {'Width': 0.0567106232047081,
     'Height': 0.14229333400726318,
     'Left': 0.8704260587692261,
     'Top': 0.482204407453537},
    'Landmarks': [{'Type': 'eyeLeft',
      'X': 0.8861061930656433,
      'Y': 0.5411022305488586},
     {'Type': 'eyeRight', 'X': 0.9124276041984558, 'Y': 0.5401004552841187},
     {'Type': 'mouthLeft', 'X': 0.8891450762748718, 'Y': 0.5897415280342102

In [5]:
final_timestamps = {}
next_token = "Y"
first_round = True

while next_token != "":
    # Query Reko Video
    if first_round:
        next_token = ""
        first_round = False
    response = reko.get_face_detection(JobId=job_id, NextToken=next_token)
    
    # Iterate over every face
    for face in response['Faces']:
        f = face["Face"]["BoundingBox"]
        t = str(face["Timestamp"])
        time_faces = final_timestamps.get(t)
        if time_faces == None:
            final_timestamps[t] = []
        final_timestamps[t].append(f)
    try:
        next_token = response['NextToken']
    except:
        break

print("Complete")

Complete


In [6]:
!pip install opencv-python -q

You should consider upgrading via the '/usr/local/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.


In [7]:
import cv2
import numpy as np
import time


In [15]:
frame_rate = response["VideoMetadata"]["FrameRate"]
frame_height = response["VideoMetadata"]["FrameHeight"]
frame_width = response["VideoMetadata"]["FrameWidth"]

frame_counter = 0
fourcc = cv2.VideoWriter_fourcc(*'XVID')

v = cv2.VideoCapture('videos/video-test.mp4')
out = cv2.VideoWriter('videos/output.mp4', fourcc, int(frame_rate), (frame_width, frame_height), True)

while v.isOpened():
    stime = time.time()
    has_frame, frame = v.read()
    if has_frame:
        for t in final_timestamps:
            faces = final_timestamps.get(t)
            if int(int(t)/1000*frame_rate) == frame_counter:
                for f in faces:
                    x = int(f['Left']*frame_width)
                    y = int(f['Top']*frame_height)
                    w = int(f['Width']*frame_width)
                    h = int(f['Height']*frame_height)
                    frame = cv2.rectangle(frame, (x,y), (x+w,y+h), (255,0,0), 3)
                out.write(frame)
    else:
        break
    frame_counter += 1
out.release()
v.release()
cv2.destroyAllWindows()
print(f"Complete. {frame_counter} frames were written.")

Complete. 2246 frames were written.


In [9]:
v = cv2.VideoCapture('videos/video-test.mp4')
print(v.get(5))
v.release()

29.97002997002997


In [ ]:
6